# Generando mapas a partir del DF_provincias.

In [1]:
#Cargando el DF que usaremos para representar datos en el mapa.

import pandas as pd

df_provincias = pd.read_csv('provincias.csv')
df_provincias.head()

,provincia,altitud,distancia,desnivel,pendiente,coeficiente
0,A CORUÑA,505,6,436,7,117
1,ALBACETE,1402,12,782,6,174
2,ALICANTE,810,7,577,7,171
3,ALMERÍA,1670,18,1001,4,216
4,ASTURIAS,1163,11,818,7,226


In [2]:
#Asignando una variable al archivo geojson de la península dividida en provincias.

import json

provincias = 'provincias.geojson'

In [3]:
#Explorando el geojson para ver de qué manera exacta nombra a las provincias.

with open(provincias, encoding='utf-8') as provincias_file:
    provincias_json = json.load(provincias_file)
    
provincias_lista = [] #Lista vacía a la que añadiremos los nombres de provincia.

for index in range(len(provincias_json['features'])):
    provincias_lista.append(provincias_json['features'][index]['properties']['texto'])

In [4]:
test_list = ['Ã\x81lava',
 'Albacete',
 'Alicante',
 'AlmerÃ\xada',
 'Ã\x81vila',
 'Badajoz',
 'Islas Baleares',
 'Barcelona',
 'Burgos',
 'CÃ¡ceres',
 'CÃ¡diz',
 'CastellÃ³n',
 'Ciudad Real',
 'CÃ³rdoba',
 'La CoruÃ±a',
 'Cuenca',
 'Gerona',
 'Granada',
 'Guadalajara',
 'Huelva',
 'Huesca',
 'JaÃ©n',
 'LeÃ³n',
 'Lleida',
 'La Rioja',
 'Lugo',
 'Madrid',
 'MÃ¡laga',
 'Murcia',
 'Navarra',
 'Orense',
 'Asturias',
 'Palencia',
 'Las Palmas',
 'Pontevedra',
 'Salamanca',
 'Santa Cruz de Tenerife',
 'Cantabria',
 'Segovia',
 'Sevilla',
 'Soria',
 'Teruel',
 'Toledo',
 'Valencia',
 'Valladolid',
 'Vizcaya',
 'Zamora',
 'Zaragoza',
 'Tarragona',
 'GuipÃºzcoa',
 'Ceuta',
 'Melilla']

In [5]:
#Vemos que las provincias no concuerdan en nombre con las del DF por culpa de los acentos y caracteres especiales.
#Las colocamos en el mismo orden que en el DF para facilitarnos la labor.

provincias_lista = ['La CoruÃ±a',
 'Albacete',
 'Alicante',
 'AlmerÃ\xada',
 'Asturias',
 'Badajoz',
 'Barcelona',
 'Vizcaya',
 'Burgos',
 'Cantabria',
 'CastellÃ³n',
'Ceuta',
 'Ciudad Real',
  'Cuenca',
'CÃ¡ceres',
 'CÃ¡diz',
 'CÃ³rdoba',
 'Gerona',
'GuipÃºzcoa',    
 'Granada',
 'Guadalajara',
 'Huelva',
 'Huesca',
 'JaÃ©n',
 'La Rioja',    
 'LeÃ³n',
 'Lleida',
 'Lugo',
 'Madrid',
 'Murcia',
'MÃ¡laga',
 'Navarra',
 'Orense',
 'Palencia',
 'Pontevedra',
 'Salamanca',
 'Segovia', 
 'Sevilla',
 'Soria',
 'Tarragona',
 'Teruel',
 'Toledo',
 'Valencia',
 'Zamora',
 'Zaragoza',
'Ã\x81lava', 
'Ã\x81vila']

#Cambiamos nuestros nombres de provincia por los nuevos.

df_provincias['provincia'] = provincias_lista

In [7]:
#Buscamos qué provincias están faltas de datos para evitar fallos en el mapa, ya que Folium rellena las áreas sin datos con
#la tonalidad más oscura posible.

lista_incompletos = []

for i in test_list:
    if i not in provincias_lista:
        lista_incompletos.append(i)
        
lista_incompletos

['Islas Baleares',
 'Las Palmas',
 'Santa Cruz de Tenerife',
 'Valladolid',
 'Melilla']

In [8]:
#Añadimos uno a uno diccionarios con los nombres de provincia faltantes pero con datos nulos.

dict_missing = {'provincia': 'Islas Baleares', 'altitud': 0, 'distancia': 0, 'desnivel': 0, 'pendiente': 0, 'coeficiente': 0}
df_provincias = df_provincias.append(dict_missing, ignore_index=True)

dict_missing = {'provincia': 'Las Palmas', 'altitud': 0, 'distancia': 0, 'desnivel': 0, 'pendiente': 0, 'coeficiente': 0}
df_provincias = df_provincias.append(dict_missing, ignore_index=True)

dict_missing = {'provincia': 'Santa Cruz de Tenerife', 'altitud': 0, 'distancia': 0, 'desnivel': 0, 'pendiente': 0, 'coeficiente': 0}
df_provincias = df_provincias.append(dict_missing, ignore_index=True)

dict_missing = {'provincia': 'Valladolid', 'altitud': 0, 'distancia': 0, 'desnivel': 0, 'pendiente': 0, 'coeficiente': 0}
df_provincias = df_provincias.append(dict_missing, ignore_index=True)

dict_missing = {'provincia': 'Melilla', 'altitud': 0, 'distancia': 0, 'desnivel': 0, 'pendiente': 0, 'coeficiente': 0}
df_provincias = df_provincias.append(dict_missing, ignore_index=True)

In [15]:
#Importamos folium y generamos la capa base comunicándole las coordenadas, nivel de zoom y paleta de colores.

import folium

mapa_provincias = folium.Map(location=[40.416775, -3.703790], zoom_start=6, tiles='stamenwatercolor')

In [16]:
#Creando el mapa.

provincias_text = json.load(open('provincias.geojson', encoding='utf-8')) #Muy importante añadir el encoding.

mapa_provincias.choropleth(
    geo_data = provincias_text,
    data = df_provincias,
    columns = ['provincia', 'coeficiente'], #Especificamos las columnas que usaremos como input.
    key_on = 'feature.properties.texto',
    fill_color='YlGnBu', 
    fill_opacity=1, 
    line_opacity=1,
    legend_name='Coeficiente medio.', #Desde este parámetro modificaremos la leyenda.
    smooth_factor=0)

In [17]:
#Finalmente, el mapa.

mapa_provincias

In [ ]:
###FINALIZADO###